In [1]:
from langchain.llms import CTransformers  # to load the llama2 model

In [2]:
llm = CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                        model_type="llama", config={'max_new_tokens':3800,'temperature': 0.5,'context_length': 3600})

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE transactions (
	`Account_No` VARCHAR(50) NOT NULL, 
	`Transaction_details` TEXT, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_amount` INTEGER, 
	`Value_date` DATE, 
	`Date` DATE
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from transactions table:
Account_No	Transaction_details	Withdrawal_amount	Deposit_amount	Balance_amount	Value_date	Date
409000611074'	TRF FROM  Indiaforensic SERVICES	0	1000000	1000000	2022-10-05	2022-10-05
409000611074'	TRF FROM  Indiaforensic SERVICES	0	1000000	2000000	2022-10-11	2022-10-11
409000611074'	FDRL/INTERNAL FUND TRANSFE	0	500000	2500000	2022-10-24	2022-10-24
*/


In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)


# FEW SHORT LEARNING

In [5]:
few_shots=[
    {
        'Question':"What is my income in my last 3 months.My Account Number is 409000493201'. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = "409000493201'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);
        """,
        'SQLResult':"16725509",
        'Answer':"16725509 is the income of last 3 months."
    },

    {
        'Question':"What is the total expenses of last 8 months for account number 409000611074' ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE 
Account_No = "409000611074'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);
        """,
        'SQLResult':"75124046",
        'Answer':"75124046 is the total expenses of last 8 months."
    },
      {
        'Question':"How much did I save last month as my account number is 409000493201'  ?",
        'SQLQuery':"""SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM 
transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() 
- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);
        """,
        'SQLResult':"-193509",
        'Answer':"You saved -193509 last month."
    },
    {
        'Question':"How many transactions did I make in last 2 week as my account number is 409000493201' ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE 
Account_No = "409000493201'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK);
        """,
        'SQLResult':"24",
        'Answer':"You made 24 transactions in last 2 week."
    },
     {
        'Question':"What was my total spending last week as my account number is 409000493201' ",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Account_No = "409000493201'" AND 
Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK);
        """,
        'SQLResult':"515559",
        'Answer':"515559 is your total spending last week."
    }
#     ,
#     {
#         'Question': "Give me a breakdown of my income and expenses for each day this month as my account number is  409000493201' ?",
#         'SQLQuery': """SELECT Date, SUM(Deposit_amount) AS Total_income ,SUM(Withdrawal_amount) AS Total_Expenses 
# FROM transactions 
# WHERE Account_No = "409000493201'"
# AND YEAR(Value_date) = YEAR(CURRENT_DATE())
# AND MONTH(Value_date) = MONTH(CURRENT_DATE())
# GROUP BY Date;
# """,
#         'SQLResult':
#         """
# Date              Total_income          Total_Expenses
# 2024-06-02         200000                 183648
# 2024-06-03         0                      80666
# 2024-06-04         200000                 135504
# 2024-06-05         0                      92031
# 2024-06-06         0                      118475
# 2024-06-07         300000                 16440
# 2024-06-09         0                      62112
# 2024-06-10         0                      10331
# """    ,
#         'Answer': """ Result:
# Date              Total_income          Total_Expenses
# 2024-06-02         200000                 183648
# 2024-06-03         0                      80666
# 2024-06-04         200000                 135504
# 2024-06-05         0                      92031
# 2024-06-06         0                      118475
# 2024-06-07         300000                 16440
# 2024-06-09         0                      62112
# 2024-06-10         0                     10331
# """         
#     }
    , {
        'Question': "Give me a breakdown of my expenses for each day this month as my account number is  409000493201' ?",
        'SQLQuery': """SELECT Date, SUM(Withdrawal_amount) AS Total_Expenses 
FROM transactions 
WHERE Account_No = "409000493201'"
AND YEAR(Value_date) = YEAR(CURRENT_DATE())
AND MONTH(Value_date) = MONTH(CURRENT_DATE())
GROUP BY Date;
""",
        'SQLResult':
        """
Date                        Total_Expenses
2024-06-02                   183648
2024-06-03                   80666
2024-06-04                   135504
2024-06-05                   92031
2024-06-06                   118475
2024-06-07                   16440
2024-06-09                   62112
2024-06-10                   10331
"""
        ,
        'Answer': """The total_expenses is 183648,80666,135504,92031,118475,16440,62112,10331 for the year 2024 month of 6 and date 02,03,04,05,06,07,09 and 10.  """         
    },
    {
        'Question':"Will it rain today ? ",
        'SQLQuery': "",
        'SQLResult':"",
        'Answer':"Sorry I dont have access to the information."
    }


]


In [6]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import json


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Convert lists and dictionaries to JSON strings
to_vectorize = [
    " ".join(
        [json.dumps(value) if isinstance(value, (list, dict)) else value for value in example.values()]
    )
    for example in few_shots
]


c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TypeError: sequence item 1: expected str instance, NoneType found

In [ ]:
to_vectorize 

['What is my income in my last 3 months.My Account Number is 409000493201\'. ? SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = "409000493201\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);\n         16725509 16725509 is the income of last 3 months.',
 'What is the total expenses of last 8 months for account number 409000611074\' ? SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE \nAccount_No = "409000611074\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);\n         75124046 75124046 is the total expenses of last 8 months.',
 'How much did I save last month as my account number is 409000493201\'  ? SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM \ntransactions WHERE Account_No = "409000493201\'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() \n- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);\n         -193509 You saved -193509 last mon

In [ ]:

# Convert metadata to JSON strings
metadatas = [
    {k: json.dumps(v) if isinstance(v, (list, dict)) else v for k, v in example.items()}
    for example in few_shots
]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=metadatas)

In [ ]:
metadatas

[{'Question': "What is my income in my last 3 months.My Account Number is 409000493201'. ?",
  'SQLQuery': 'SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = "409000493201\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);\n        ',
  'SQLResult': '16725509',
  'Answer': '16725509 is the income of last 3 months.'},
 {'Question': "What is the total expenses of last 8 months for account number 409000611074' ?",
  'SQLQuery': 'SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE \nAccount_No = "409000611074\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);\n        ',
  'SQLResult': '75124046',
  'Answer': '75124046 is the total expenses of last 8 months.'},
 {'Question': "How much did I save last month as my account number is 409000493201'  ?",
  'SQLQuery': 'SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM \ntransactions WHERE Account_No = "409000493201\'" AND YEAR(Value_date)

In [ ]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)
example_selector.select_examples({"Question": "Do dogs bark?"})

[{'Answer': 'Sorry I dont have access to the information.',
  'Question': 'Will it rain today ? '}]

In [ ]:
# Define the custom prompt for SQL database interactions
custom_mysql_prompt = """You are an expert in converting natural language into MySQL queries. Follow these instructions carefully to ensure accuracy:
1. Only use the columns specified in the table information provided below.
2. Before proceeding, check if the columns mentioned in the user's natural language query are found in the database. If not, respond with "Sorry, I don't have access to the information."
3. If the columns mentioned in the user's natural language are not found, send the response "Sorry, no information available" and abort/exit without proceeding further.
4. Use the CURDATE() function to get the current date when the question involves "today," "last month," "last year," or "last week."
5. Ensure that your queries are syntactically correct and optimized for performance.
6. Provide clear, accurate, and concise SQL queries without making assumptions beyond the given data.

Table information:
1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

Use queries for at most {top_k} results.
Remember to only use the provided table columns and structure your queries to handle the specified requests accurately."""


In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

In [ ]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [ ]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [ ]:
suffix_eg="Question: {input}"

In [ ]:
print(suffix_eg)

Question: {input}


In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=custom_mysql_prompt,
    suffix=suffix_eg,
    input_variables=["input", "top_k"], #These variables are used in the prefix and suffix
)

In [ ]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [ ]:
print(few_shot_prompt)

input_variables=['input', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000017BB8F23D10>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Question: {input}' prefix='You are an expert in converting natural language into MySQL queries. Follow these instructions carefully to ensure accuracy:\n1. Only use the columns specified in the table information provided below.\n2. Before proceeding, check if the columns mentioned in the user\'s natural language query are found in the database. If not, respond with "Sorry, I don\'t have access to the information."\n3. Use the CURDATE() function to get the current date when the question involves "today," "last month," "last year," or "last week."\n4.

In [ ]:
# result = new_chain("What was my total earning last week as my account number is 409000493201'")

In [ ]:
new_chain("Do cat bark?")



> Entering new SQLDatabaseChain chain...
Do cat bark?
SQLQuery:

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# print(result)

In [ ]:
# query = result['query']
# sql_result = result['result']
# # Define a function to generate the final answer
# def generate_final_answer(query, sql_result):
#     prompt = f"The result of the query '{query}' is '{sql_result}'. Please provide a complete answer based on this result."
#     return llm.generate([prompt])



In [ ]:
# # Use the LLM to generate the final answer
# final_answer = generate_final_answer(query, sql_result)
# print(final_answer)